In [ ]:
import pandas as pd

Checking an example csv for heart rate


In [ ]:
df_ex_hr=pd.read_csv('datasets/COVID-19-Wearables/A0KX894_hr.csv')
df_ex_hr.head(10)

In [ ]:
df_ex_hr.describe()

In [ ]:
df_ex_hr.info()

In [ ]:
df_ex_hr['datetime'].describe()

Max heart rate is normally 220 - 20, as the data recorded complies with the standards of Stanford universities, the participants should be around or superior to 20. lets do a quick check on one of the recorded data


In [ ]:
nan_entries = df_ex_hr.isna().sum()
illogical_entries = df_ex_hr[(df_ex_hr['heartrate'] < 30)| (df_ex_hr['heartrate'] > 200)].sum()
print('Number of NaN entries: ', nan_entries)
print('Number of illogical entries: ', illogical_entries)

In [ ]:
df_ex_steps=pd.read_csv('datasets/COVID-19-Wearables/A0KX894_hr.csv')
df_ex_steps.head(10)

In [ ]:
df_ex_steps.describe()

In [ ]:
df_ex_steps.info()

In [ ]:
df_ex_steps['datetime'].describe()

Lets ittereate over the directories and save a dictionary containing the users name and the corresponding hr and steps

In [ ]:
import os
dir_csv = 'datasets/COVID-19-Wearables/'
files = [f for f in os.listdir(dir_csv) if f.endswith('.csv')] # we can straight do it with os.listdir
# noticed there are some files with user name containing_1 or _2, so we need to put that into consideration
#user_name = [f.split('_')[0] if f.count("_")==1 else ""for f in files]
# will deal with it inside the dictionary better
user_name = [f.split('_')[0] for f in files]
mult=0
for i in user_name:
    if(user_name.count(i)) < 1:
        print('missing data for user: ', i)
    if(user_name.count(i)) > 2:
        mult+=1
       
user_name_unique = list(set(user_name))

dict_user_data={}   
for i in user_name_unique:
    dict_user_data[i] = dict()# I am not concatinating strings, there is a user with multi entries
    dict_user_data[i]['hr'] = [f for f in files if f == i+'_hr.csv']
    dict_user_data[i]['steps'] = [f for f in files if f == i+'_steps.csv']
    s=[f for f in files if f == i+'_sleep.csv']
    dict_user_data[i]['sleep'] = s if len(s)!=0 else None
    if files.count(i+'_steps_longterm.csv.csv')>0:
        dict_user_data[i]['steps_longterm']=i+'_steps_longterm.csv'
    if files.count(i+'_hr_longterm.csv.csv')>0:
        dict_user_data[i]['hr_longterm']=i+'_hr_longterm.csv'
    if files.count(i+'_sleep_longterm.csv.csv')>0:
        dict_user_data[i]['sleep_longterm']=i+'_sleep_longterm.csv'

import json
with open('datasets/dict_user_data.json', 'w') as fp:
    json.dump(dict_user_data, fp)
    
# to visualize all the directory structure


In [ ]:
merged_df_same_user_dict = {}
merged_df_same_user_dict_2 = {}

for user in dict_user_data.keys():
    if dict_user_data[user]['hr'] != []:
        df_hr = pd.read_csv(dir_csv + dict_user_data[user]['hr'][0])
        df_hr['datetime'] = pd.to_datetime(df_hr['datetime'])
        df_hr.set_index('datetime', inplace=True)
        df_hr=df_hr.resample('1Min').mean() # resample to 1 min
        #df_hr['heartrate'].resample('1min').mean()
    if dict_user_data[user]['steps'] != []:
        df_steps=pd.read_csv(dir_csv + dict_user_data[user]['steps'][0])
        df_steps['datetime'] = pd.to_datetime(df_steps['datetime'])
        df_steps.set_index('datetime', inplace=True)
        df_steps =df_steps.resample('1Min') 
    merged_df_same_user_dict[user]=pd.merge(df_hr, df_steps, on="datetime", how="left")
    #merged_df_same_user_dict_2[user]=pd.merge(df_hr, df_steps, left_index=True, right_index=True, how="outer")
    



In [ ]:
# kenerl doesnt work, so dividing the steps into 2 parts
for user in dict_user_data.keys():
    merged_df_same_user_dict[user].fillna(method='ffill', inplace=True)
    merged_df_same_user_dict[user]=merged_df_same_user_dict[user].rename(columns={ "user_x": "user"})
    merged_df_same_user_dict[user]=merged_df_same_user_dict[user].drop(columns=['user_y'], axis=1)
    merged_df_same_user_dict[user]=merged_df_same_user_dict[user].dropna()
    


In [ ]:
#testing for specific instance

In [ ]:
df_t = merged_df_same_user_dict['A0KX894']

In [ ]:
df_t.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig=plt.figure(figsize=(20,10))
ax=fig.add_subplot(1,1,1)
ax2=ax.twinx()
ax.plot(df_t['heartrate'])
ax2.plot(df_t['steps'], color='r')
ax.set_ylabel('heartrate')
ax2.set_ylabel('steps')


In [ ]:
from math import gamma
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
fig=plt.figure(figsize=(20,10))
ax=fig.add_subplot(1,1,1)
ax2=ax.twinx()
ax.scatter(df_t.index,df_t['steps'],color='r',alpha=1) # different unit we know we dont do an ax twinx temporarily just to visualize correlation
ax2.scatter(df_t.index,df_t['heartrate'],alpha=0.1)
ax.set_ylabel('steps')
ax2.set_ylabel('heartrate')

In [ ]:
df_t.head()

In [ ]:
import numpy as np
sec_t=60 * 24 * 60
df_t['totalsec_newday']=pd.to_timedelta(df_t.index.strftime('%H:%M:%S')).total_seconds()
df_t['sin_time'] = np.sin(2*np.pi*df_t['totalsec_newday']/sec_t)
df_t['cos_time'] = np.cos(2*np.pi*df_t['totalsec_newday']/sec_t)
df_t.head(1000)

In [ ]:
df_t.sample(50).plot.scatter('sin_time','cos_time').set_aspect('equal')

In [ ]:
df_test=df_t.reset_index(level=0)
df_test['index1'] = df_t.index
corrMatrix = df_test.corr()
sns.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
# we can already exclude features due to high correlation but not yet untill merge 
# sin and cos look promising 
for user in dict_user_data.keys():
    merged_df_same_user_dict[user]['totalsec_newday']=pd.to_timedelta(merged_df_same_user_dict[user].index.strftime('%H:%M:%S')).total_seconds()
    merged_df_same_user_dict[user]['sin_time'] = np.sin(2*np.pi*merged_df_same_user_dict[user]['totalsec_newday']/sec_t)
    merged_df_same_user_dict[user]['cos_time'] = np.cos(2*np.pi*merged_df_same_user_dict[user]['totalsec_newday']/sec_t)
    merged_df_same_user_dict[user]['time'] = merged_df_same_user_dict[user].index

In [ ]:
df_master = pd.concat(merged_df_same_user_dict.values(), ignore_index=True)

In [ ]:
df_master.head()

In [ ]:
df_master["user"].unique() # confirmed all users are in the master df

In [ ]:
df_master.to_csv('datasets/df_master.csv')

In [ ]:
len(df_master['user'].values)